In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as colors
pio.templates.default = "plotly_white"

In [2]:
df = pd.read_csv('cafe_sales.csv')

In [3]:
df.shape

(10000, 8)

In [4]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [5]:
df.tail()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02
9999,TXN_6170729,Sandwich,3,4.0,12.0,Cash,In-store,2023-11-07


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [8]:
# need to change columns to lowercase and put _ where space is
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df.head()

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [9]:
df.isnull().sum()

,0
transaction_id,0
item,333
quantity,138
price_per_unit,179
total_spent,173
payment_method,2579
location,3265
transaction_date,159


In [10]:
# # Handle Missing Values:

# # Replace missing categorical values with the mode.

replace_with_mode = ['item', 'payment_method', 'location']

for col in replace_with_mode:
    mode_value = df[col].mode()[0]  # Find mode of the column
    df[col] = df[col].replace(to_replace=r'(?i)(error|unknown)', value=mode_value, regex=True)



for col in replace_with_mode:
    mode_value = df[col].mode()[0]  # Find mode of the column
    df[col] = df[col].replace(['', ' ', None], mode_value)


df.isnull().sum()

,0
transaction_id,0
item,0
quantity,138
price_per_unit,179
total_spent,173
payment_method,0
location,0
transaction_date,159


In [11]:
# Fill missing numeric values with the median.

numeric_columns = ['quantity', 'price_per_unit']

for col in numeric_columns:
    # Convert column to numeric (coerce non-numeric values to NaN)

    df[col] = pd.to_numeric(df[col], errors='coerce')

    # Calculate the median of the column, ignoring NaN

    median_value = df[col].median()

    # Replace NaN (including blanks and strings) with the median value

    df[col] = df[col].fillna(median_value)


In [12]:
df.isnull().sum()

,0
transaction_id,0
item,0
quantity,0
price_per_unit,0
total_spent,173
payment_method,0
location,0
transaction_date,159


In [13]:
# Replace invalid values in 'total_spent' with the product of 'quantity' and 'price_per_unit'

df['total_spent'] = pd.to_numeric(df['total_spent'], errors='coerce')  # Coerce errors to NaN
df['total_spent'] = df['total_spent'].fillna(df['quantity'] * df['price_per_unit']) # Replace NaN with calculated value

In [14]:
df.isnull().sum()

,0
transaction_id,0
item,0
quantity,0
price_per_unit,0
total_spent,0
payment_method,0
location,0
transaction_date,159


In [15]:
# handle with blanks in transaction date column

# Convert 'date' column to datetime, invalid parsing will turn into NaT

df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')

# Remove rows where 'date' column is NaT or blank

df = df[df['transaction_date'].notna()]

In [16]:
df.isnull().sum()

,0
transaction_id,0
item,0
quantity,0
price_per_unit,0
total_spent,0
payment_method,0
location,0
transaction_date,0


In [17]:
df.shape

(9540, 8)

In [18]:
df.duplicated().sum()

0

In [19]:
df['month'] = df['transaction_date'].dt.strftime('%b')
df['year'] = df['transaction_date'].dt.year
df['day'] = df['transaction_date'].dt.day
df['day_of_week'] = df['transaction_date'].dt.day_name()

In [20]:
  df.describe()

,quantity,price_per_unit,total_spent,transaction_date,year,day
count,9540.000000,9540.000000,9540.000000,9540,9540.0,9540.000000
mean,3.024109,2.952673,8.924109,2023-07-01 23:00:31.698113536,2023.0,15.738679
min,1.000000,1.000000,1.000000,2023-01-01 00:00:00,2023.0,1.000000
25%,2.000000,2.000000,4.000000,2023-04-01 00:00:00,2023.0,8.000000
50%,3.000000,3.000000,8.000000,2023-07-02 00:00:00,2023.0,16.000000
75%,4.000000,4.000000,12.000000,2023-10-02 00:00:00,2023.0,23.000000
max,5.000000,5.000000,25.000000,2023-12-31 00:00:00,2023.0,31.000000
std,1.384186,1.243449,5.997956,NaN,0.0,8.761003


In [35]:
# Now let’s have a look at the monthly sales

sales_by_month = df.groupby('month')['total_spent'].sum().reset_index()

fig = px.line(sales_by_month, x='month', y='total_spent', title='Monthly Sales', width=1000)
fig.show()

In [22]:
# Now let’s have a look at the sales by category:

sales_by_category = df.groupby('item')['total_spent'].sum().reset_index().sort_values(by='total_spent', ascending=False)

fig = px.bar(sales_by_category, x='item', y='total_spent', title='Sales by Category', color='item' ,width=1000)
fig.show()


In [30]:
# Customer Behavior by Payment Method

payment_behavior = df.groupby('payment_method')['total_spent'].agg(['sum', 'mean', 'count'])
payment_behavior

,sum,mean,count
payment_method,,,
Cash,19588.0,9.076923,2158
Credit Card,19522.5,8.996544,2170
Digital Wallet,46025.5,8.830679,5212


In [23]:
# Sales by payment method

sales_by_payment_method = df.groupby('payment_method')['total_spent'].sum().reset_index()

fig = px.pie(sales_by_payment_method, values='total_spent', names='payment_method', hole=0.5, title='Sales by Payment Method', width=1000)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()


In [34]:
# Identify the top 5 popular items
popular_items = df.groupby('item')['quantity'].sum().sort_values(ascending=False).head(5)

fig = px.bar(popular_items, x=popular_items.index, y='quantity', title='Most Popular Items', color=popular_items.index, width=1000)
fig.show()

In [25]:
# Sales Distribution by Location

location_sales = df.groupby('location')['total_spent'].sum().reset_index()

fig = px.pie(location_sales, values='total_spent', names='location', hole=0.5, title='Sales Distribution by Location', width=1000)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig

In [26]:
# Most Profitable Days

sales_by_day = df.groupby('day_of_week')['total_spent'].sum()
sales_by_day

,total_spent
day_of_week,
Friday,12374.5
Monday,12171.0
Saturday,12067.5
Sunday,12309.5
Thursday,12448.0
Tuesday,12071.0
Wednesday,11694.5
